In [ ]:
!pip

# Exctracting frames


In [ ]:
!sudo apt install tesseract-ocr

In [ ]:
!pip install pytesseract
# !pip install -U Pillow

!pip install Pillow==7.1.2


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('/content/train_comb - train_comb (3).csv')
# data.shape
# data = data[data["id"].str.contains("\*") == False]

data['comb'] = data['employee'].astype(str) + '/' + data['action'].astype(str)

# for row in data.iterrows():
#   print(row)


In [ ]:
del data

In [ ]:
count_in_classes = pd.DataFrame(data['comb'].value_counts(ascending=True))
# print(type(count_in_classes))
for cl in count_in_classes.iterrows():
  print(cl[0], cl[1][0])

n, bins, patches = plt.hist(data['comb'], 50, density=True, facecolor='g', alpha=0.75)

In [ ]:

date_train = data['date'].values
print(len(data))

3678


In [ ]:
import os
dir_path = r'/content/gdrive/MyDrive/DigitalGap/validation'

already_exctracted = []

for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        already_exctracted.append(path.split('.')[0])
print(len(already_exctracted))

new_data = data[data.id.isin(list(map(int, already_exctracted))) == False]
print(len(new_data))

date_train = new_data['date'].values
print(new_data['id'].values)


0
3519
[1256 1257 1258 ... 4772 4773 4774]


In [ ]:
import shutil
import random 


random.shuffle(already_exctracted)
cnt = 0
for im in already_exctracted:
  if cnt == 792:
    break
  try:
    shutil.move(f'/content/gdrive/MyDrive/DigitalGap/validation/{im}.jpg', '/content/gdrive/MyDrive/DigitalGap/tr1_train') 
    cnt += 1

  except Exception:
    continue


In [ ]:
import pytesseract
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt

WHITE = [255,255,255]
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/DigitalGap/train2.avi')
frameNr = 15
extracted_date = []
extracted_id = []

while (2):
  success,image = vidcap.read()
  if frameNr % 15 == 0:
    if success:
      frame_re = image[50:200, 0:800]
      # (thresh, blackAndWhite60) = cv2.threshold(frame_re, 20, 255, cv2.THRESH_BINARY)

      # constant60= cv2.copyMakeBorder(blackAndWhite60,20,20,20,20,cv2.BORDER_CONSTANT,value=WHITE)

      extractedInformation60 = pytesseract.image_to_string(frame_re)
      
      date60 = extractedInformation60[1:10].replace('-','/').strip()

      time60 = extractedInformation60[15:23].strip()

      full_date60 = date60 + " " + time60

      if not(full_date60 in extracted_date) and (full_date60 in date_train):
        print(time60)
        extracted_date.append(full_date60)
        id = data.loc[data['date'] == full_date60]['id'].values[0]
        cv2.imwrite(f'/content/gdrive/MyDrive/DigitalGap/tr1_train/{id}.jpg' , image)
        extracted_id.append(id)
      frameNr += 1
    else:
      break
  else:
    frameNr += 1
    continue

with open('res.txt', 'w') as fp:
    for item in extracted_date:
        fp.write("%s\n" % item)

files.download( "res.txt" ) 



11:14:48
11:48:12
11:52:28
13:22:08
13:26:48
14:50:38


# Новый раздел

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense,Flatten,MaxPool2D
from keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
# from keras.callbacks import History
# from keras.callbacks import ModelCheckpoint, TensorBoard
# from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation
# from keras.losses import categorical_crossentropy
# from keras.models import Model, Input
# from tensorflow.keras.optimizers import Adam
# from tensorflow.python.framework.ops import Tensor
# from typing import Tuple, List



np.random.seed(1)

# print(keras.__version__)

In [ ]:
print(data[data['id']== str(18517)])


         id                date  employee  action  comb
3735  18517  5/25/2022 06:53:25         7      15  7/15


In [ ]:
train_images = []       
train_labels = []
shape = (512,512)  
train_path = '/content/gdrive/MyDrive/DigitalGap/train2'
compare_dict = {}
label = ''
for filename in os.listdir(train_path):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(train_path,filename))

        # Spliting file names and storing the labels for image in list
        
        id = filename.split('.')[0]
        if len(data[data['id']==str(id)]['employee'].values) > 0:
          label = data[data['id']==str(id)]['employee'].values[0]

        train_labels.append(label)
        # print(id, label)
        # Resize all images to a specific shape
        img = cv2.resize(img,shape).reshape(512,512,3)

        train_images.append(img)
        compare_dict[label] = img

print(len(train_labels))
# Converting labels into One Hot encoded sparse matrix




1975


In [ ]:
for img in train_images:
  print(img.shape)

In [ ]:
def augmentation_handle(img, label):
  #Augmentation
  img = img.reshape(400,400,3)
  for i in range(5):
    seed = (i, 0)  # tuple of size (2,)
    stateless_random_brightness = tf.image.stateless_random_brightness(img, max_delta=0.2, seed=seed)
    stateless_random_contrast = tf.image.stateless_random_contrast(img, lower=0.1, upper=0.9, seed=seed)
    train_images.append(stateless_random_brightness)
    train_images.append(stateless_random_contrast)
    train_labels.append(label)
    train_labels.append(label)

    stateless_random_crop = tf.image.central_crop(img, central_fraction=0.7)
    scaled_img = cv2.resize(np.float32(stateless_random_crop),(600,600))
    train_images.append(scaled_img)
    train_labels.append(label)
    
    # grayscaled = tf.image.rgb_to_grayscale(img)
    # scaled_img = cv2.resize(np.float32(grayscaled),(600,600))
    # img = grayscaled.reshape(600,600,3)
    
    # train_images.append(img)
    # train_labels.append(label)
    # print(grayscaled.shape)
    # print(len(train_labels))

In [ ]:
count_in_classes_train =  pd.DataFrame(pd.DataFrame(train_labels).value_counts(ascending=True))
less_classes = count_in_classes_train[count_in_classes_train[0] < 5]
id_for_expand = []
for row in less_classes.iterrows():
  id_for_expand.append(row[0][0])
  # print(f'{row[0][0]} -- {row[1][0]}')


for item in id_for_expand:
  if item in compare_dict:
    # print(f'For - {item}')
    augmentation_handle(compare_dict[item], item)


# print(len(train_labels))

In [ ]:
len(train_images)

1948

In [ ]:
one_hot_train_labels = pd.get_dummies(train_labels).values

# Converting train_images to array
train_images = np.array(train_images)
# Splitting Training data into train and validation dataset
x_train,x_val,y_train,y_val = train_test_split(train_images,one_hot_train_labels,random_state=42, test_size=0.33)

# print(type(x_train))

In [ ]:
del train_images
del train_labels
del data
del one_hot_train_labels

In [ ]:
count_in_classes_train =  pd.DataFrame(pd.DataFrame(train_labels).value_counts(ascending=True))
for item in count_in_classes_train.iterrows():
  print(item[0][0], item[1][0])

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

In [ ]:


# for i in range(3):
#   seed = (i, 0)  # tuple of size (2,)
#   stateless_random_contrast = tf.image.stateless_random_contrast(
#        train_images[i], lower=0.1, upper=0.9, seed=seed)
#   visualize( train_images[i], stateless_random_contrast)

# for i in range(3):
#   seed = (i, 0)  # tuple of size (2,)
#   stateless_random_crop = tf.image.central_crop(train_images[i], central_fraction=0.7)
#   visualize(train_images[i], stateless_random_crop)

# for i in range(3):
#   seed = (i, 0)  # tuple of size (2,)
#   grayscaled = tf.image.rgb_to_grayscale(train_images[i])
#   visualize(train_images[i], tf.squeeze(grayscaled))
  

#------------------------------------------
# for i in range(3):
#   seed = (i, 0)  # tuple of size (2,)
#   flipped = tf.image.flip_left_right(train_images[i])
#   visualize(train_images[i], flipped)





In [ ]:
# Visualizing Training data
print(len(train_labels))
# plt.imshow(train_images[0])

1948


In [ ]:
import tensorflow as tf

base_model = tf.keras.applications.EfficientNetV2L(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(512,512,3)
                     )

base_model.trainable=False

model = Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        # tf.keras.layers.Dropout(.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(11, activation='softmax')
    ])

# optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(
              loss='categorical_crossentropy', 
              metrics=['acc'],
              optimizer='adam'
             )
# Model Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 16, 16, 1280)     117746848 
 l)                                                              
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 1280)     8960      
 hNormalization)                                                 
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 128)               163968    
                                                                 
 dense_9 (Dense)             (None, 256)               33024     
                                                      

In [ ]:
# base_model = tf.keras.applications.EfficientNetV2L(
#                      include_top=False,
#                      weights='imagenet',
#                      input_shape=(512,512,3)
#                      )

inputs = tf.keras.Input(shape=(512, 512, 3), name="img")
x = layers.Conv2D(64, 3, activation="relu")(inputs)
x = layers.Conv2D(128, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])
x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])
x = layers.Conv2D(128, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(122, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name="toy_resnet")

# model = Sequential([
#         base_model,  
#         tf.keras.layers.BatchNormalization(renorm=True),
#         tf.keras.layers.GlobalAveragePooling2D(),
#         model1
#     ])

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_35 (Conv2D)             (None, 510, 510, 64  1792        ['img[0][0]']                    
                                )                                                                 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 508, 508, 12  73856       ['conv2d_35[0][0]']              
                                8)                                                       

In [ ]:
history = model.fit(x_train,y_train,epochs=35,batch_size=25,validation_data=(x_val,y_val))

Epoch 1/35
53/53 [==============================] - 162s 2s/step - loss: 2.3217 - acc: 0.1610 - val_loss: 2.2767 - val_acc: 0.1779
Epoch 2/35
53/53 [==============================] - 119s 2s/step - loss: 2.1965 - acc: 0.1988 - val_loss: 2.1185 - val_acc: 0.2239
Epoch 3/35
53/53 [==============================] - 119s 2s/step - loss: 2.0992 - acc: 0.1973 - val_loss: 1.9265 - val_acc: 0.2531
Epoch 4/35
53/53 [==============================] - 120s 2s/step - loss: 1.9011 - acc: 0.2902 - val_loss: 1.7060 - val_acc: 0.3712
Epoch 5/35
53/53 [==============================] - 119s 2s/step - loss: 1.7112 - acc: 0.3885 - val_loss: 1.6168 - val_acc: 0.4172
Epoch 6/35
53/53 [==============================] - 119s 2s/step - loss: 1.5706 - acc: 0.4528 - val_loss: 1.4435 - val_acc: 0.4908
Epoch 7/35
53/53 [==============================] - 121s 2s/step - loss: 1.4440 - acc: 0.5155 - val_loss: 1.3500 - val_acc: 0.5061
Epoch 8/35
53/53 [==============================] - 121s 2s/step - loss: 1.2586 - a

In [ ]:
del x_train
del x_val
del y_train
del y_val
del history
del data

NameError: ignored

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

model_1 = load_model('/content/acc94img1200shape600epoch180.h5')
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='name_of_model_1')

model_2 = load_model('/content/acc97img420shape600epoch50.h5')
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='name_of_model_2')

models = [model_1, model_2]
model_input = Input(shape=(600, 600, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')
model.save(f'ensemble.h5')

OSError: ignored

In [ ]:
file_name = '92-45comb'

model.save(f'{file_name}.h5')

In [ ]:
del model

In [ ]:
from google.colab import files
files.download(f"{file_name}.h5" ) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.models import load_model


test_images = []       
test_id = []
shape = (512,512)  
test_path = '/content/gdrive/MyDrive/DigitalGap/test'

for filename in os.listdir(test_path):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(test_path,filename))
        id = filename.split('.')[0]
        test_id.append(int(id))
        img = cv2.resize(img,shape)
        test_images.append(img)


print(test_id)

[256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455,

In [ ]:
# model_emp = load_model(f'/content/emp98.h5')
# model_act = load_model(f'/content/act95.h5')
model_comb = load_model(f'/content/91-49comb.h5')
y_test_emp = []
y_test_act = []
data_test = pd.read_csv('/content/gdrive/MyDrive/DigitalGap/test.csv')

for row in data_test.iterrows():
  id = row[1][0]
  if id in test_id:
    # y_pred_employee = model_emp.predict(test_images[test_id.index(id)].reshape(1,600, 600, 3))
    # y_pred_act = model_act.predict(test_images[test_id.index(id)].reshape(1,600, 600, 3))
    y_pred_comb = model_comb.predict(test_images[test_id.index(id)].reshape(1,512, 512, 3))

    pred_comb = np.argmax(y_pred_comb)
    # pred_emp = np.argmax(y_pred_employee)
    # pred_act = np.argmax(y_pred_act)
    # y_test_emp.append(train_labels[pred_emp])
    # y_test_act.append(train_labels[pred_act])
    y_test_emp.append(train_labels[pred_comb].split('/')[0])
    y_test_act.append(train_labels[pred_comb].split('/')[1])

data_test['employee'] = y_test_emp
data_test['action'] = y_test_act

data_test.set_index('id')
data_test.drop(['date'], axis=1, inplace=True)

data_test.reset_index(drop=True, inplace=True)
data_test

In [ ]:
from google.colab import files
data_test.to_csv(f'{file_name}.csv', encoding = 'utf-8-sig',index=False) 
files.download(f'{file_name}.csv')